In [ ]:
import logging
logging.captureWarnings(True)
logging.getLogger('py.warnings').setLevel(logging.ERROR)

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Image, display
from dask.distributed import Client, progress
from datetime import datetime, timedelta
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker
import matplotlib.path as mpath
import matplotlib.patheffects as PathEffects
from scipy import stats
import scipy
import numpy.ma as ma
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1 import ImageGrid

### Get list of days that AR made landfall in SB (lat=34.5, lon=-119.375)

In [ ]:
latitude=34.5
longitude=359.375-119.375

filename =  '../data/ar_catalog/globalARcatalog_MERRA2_1980-2017_v3.0.nc'
ds = xr.open_dataset(filename, engine='netcdf4')
ds = ds.sel(lat=latitude, lon=longitude)
ds = ds.squeeze()
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
ds.info()

In [ ]:
da = ds.shape.dropna(dim='time') ## drop the times where there is no AR present
da['time'] = da.indexes['time'].normalize() ## normalize the time index to get list of days
idx = np.unique(da['time']) ## list of days where AR is present in loc
times_list = pd.DatetimeIndex(idx) ## create a pandas datetime index for easy time use
day_list = pd.to_datetime(idx) + timedelta(hours=9)
day_list

In [ ]:
## Get only March AR days
march_ar = times_list.month ==3
march_ar = times_list[march_ar]
march_ar

In [ ]:
## generate list of times where the day is 1 - for loading the datasets
days = np.ones(len(times_list))
yearmonth_df = pd.DataFrame({'year': times_list.year, 'month': times_list.month,
                             'day': days})

yearmonth_list = pd.to_datetime(yearmonth_df)
print(len(yearmonth_list))
print(len(yearmonth_list.unique()))
yearmonth_list = pd.to_datetime(yearmonth_list.unique())
yearmonth_list

### Load only AR days of MERRA2 Data for:
- IVT (U, V, Q 1000-300 hPa)
- 0 C isotherm (T, H )

In [ ]:
# client = Client(processes=True, n_workers=3)
# client

In [ ]:
times_list = yearmonth_list # all AR days in SB
# times_list = march_ar # AR days in March
## create explicit list of files because they are saved as monthly files

yearmonth = []
file_list = []

for t in np.arange(len(times_list)):      
    y = str(times_list[t].year)
    if times_list[t].month < 10:
        month = '0' + str(times_list[t].month)
    else:
        month = str(times_list[t].month)

    yearmonth.append(y + month)
     
    file_list.append('../data/M2I6NPANA.5.12.4/global_daily_concat/' +  'out.MERRA2.inst6_3d_ana_Np.' + yearmonth[t] +'.nc')

In [ ]:
## import MERRA2 0*C isotherm data (T, H) only for AR days
def preprocess(ds):
    '''keep only the selected location and variables'''
    unused_variables = ['U', 'V', 'QV', 'SLP', 'O3', 'PS']
    ds = ds.drop(unused_variables)
    return ds.sel(lat=slice(30, 40), lon=slice(-125.0, -115.0), bnds=1, lev=slice(1000.0, 200.0))

ds = xr.open_mfdataset(file_list,
                       engine='netcdf4', concat_dim='time', autoclose=True, preprocess=preprocess)

print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
ds

In [ ]:
ds = ds.load()

In [ ]:
## Select specific days from ds
ds = ds.sel(time=day_list)
print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))

In [ ]:
save_path = '../data/MERRA-2/SB_AR_0_degree_isotherm.nc'
ds.to_netcdf(path=save_path, mode = 'w', format='NETCDF4')

In [ ]:
## import MERRA2 IVT vertical level data (U, V, QV) only for AR days
def preprocess(ds):
    '''keep only the selected location and variables'''
    unused_variables = ['T', 'H', 'SLP', 'O3', 'PS']
    ds = ds.drop(unused_variables)
    return ds.sel(lat=34.5, lon=-119.375, bnds=1, lev=slice(1000.0, 300.0))

ds = xr.open_mfdataset(file_list,
                       engine='netcdf4', concat_dim='time', autoclose=True, preprocess=preprocess)

print('ds size in GB {:0.2f}\n'.format(ds.nbytes / 1e9))
ds.info()

In [ ]:
save_path = '../data/MERRA-2/SB_march_AR_vertical_vapor_flux.nc'
ds.to_netcdf(path=save_path, mode = 'w', format='NETCDF4')